In [171]:
import pandas as pd
import numpy as np
import xarray as xr
import sgkit as sg

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df_linreg = pd.read_csv('../results/regression/linear_regression.csv')
df_logreg = pd.read_csv('../results/regression/logistic_regression.csv')
linreg_top50 = df_linreg.sort_values('pvalue')[:50]
logreg_top50 = df_logreg.sort_values('pvalue')[:50]
pd.merge(linreg_top50, logreg_top50, how='inner', on=['pos', 'alt.allele'], suffixes=['_lin', '_log'], validate='one_to_one');

In [191]:
ds_linreg = sg.load_dataset('../results/zarrs/Hinf_norm_mic_linreg_results.zarr/')
df_other = (ds_linreg[['variants', 'variant_position', 'variant_quality', 'pvalue', 'p_values.adj', 'mutation', 'r.squared', 'adj.r.squared', 'effect', 'gene.name', 'gene.product', 'gene.type']]
            .to_pandas()
            .assign(mutation_type= lambda df: ((df['mutation'] == '') | (df['mutation'].str.split().str[0].str[0] == df['mutation'].str.split().str[0].str[-1]))
                    .apply(lambda x: 'synonymous' if x else 'nonsynonymous'))
)
df_geno = ds_linreg.set_index({'samples': 'sample_id'}).call_genotype.squeeze().to_pandas()


In [4]:
import pandas as pd
pd.read_csv('../results/linreg_logscaled.csv', sep='\t')

,variants,variant_position,variant_quality,pvalue,p_values.adj,mutation,r.squared,adj.r.squared,effect,gene.name,...,HLR-561,HLR-562,HLR-563,HLR-565,HLR-566,HLR-568,HLR-569,HLR-571,HLR-572,HLR-573
0,ID_1688582_C,1688582,17172.80,1.262156e-26,1.862880e-21,T513T (ACA/ACG),3.524091e-01,0.349947,1.588410,ftsI CDS,...,0,1,0,0,0,0,1,0,0,0
1,ID_1688849_G,1688849,22054.80,1.182945e-18,8.729837e-14,N424N (AAT/AAC),2.562792e-01,0.253451,1.325293,ftsI CDS,...,0,0,0,0,0,0,0,0,0,0
2,ID_1688876_A,1688876,21891.80,4.825926e-18,2.374275e-13,G415G (GGA/GGT),2.483717e-01,0.245514,1.340451,ftsI CDS,...,0,0,0,0,0,0,0,0,0,0
3,ID_1688588_C,1688588,18706.80,6.435606e-18,2.374658e-13,V511V (GTA/GTG),2.467430e-01,0.243879,1.388482,ftsI CDS,...,1,1,0,1,0,0,1,0,0,1
4,ID_1366000_A,1366000,26901.80,1.701882e-17,2.790992e-13,NaN,2.412153e-01,0.238330,1.444945,usg CDS,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147590,ID_326826_G,326826,6630.77,9.999547e-01,9.999656e-01,NaN,1.226561e-11,-0.003802,0.000019,glpT CDS,...,0,0,0,0,0,0,0,0,0,0
147591,ID_615265_G,615265,16479.80,9.999547e-01,9.999656e-01,H169H (CAT/CAC),1.226561e-11,-0.003802,0.000019,yegQ CDS,...,0,0,0,0,0,0,0,0,0,0
147592,ID_1745660_GA,1745660,46031.70,9.999571e-01,9.999656e-01,NaN,1.103163e-11,-0.003802,0.000009,NaN,...,1,0,1,1,1,0,1,1,1,1
147593,ID_1074841_T,1074841,18353.80,9.999588e-01,9.999656e-01,T286T (ACG/ACA),1.014344e-11,-0.003802,0.000010,prfC CDS,...,0,0,0,0,0,0,0,0,0,0


In [194]:
df_other[['mutation', 'mutation_type']].sort_values('mutation_type')

,mutation,mutation_type
variants,,
ID_892684_T,T165N (ACT/AAT),nonsynonymous
ID_1419104_T,V1005I (GTA/ATA),nonsynonymous
ID_1419155_C,I988V (ATT/GTT),nonsynonymous
ID_1419168_A,E983D (GAA/GAT),nonsynonymous
ID_1419172_G,N982T (AAT/ACT),nonsynonymous
...,...,...
ID_641290_C,L86L (TTG/CTG),synonymous
ID_641286_G,E84E (GAA/GAG),synonymous
ID_641277_T,S81S (AGC/AGT),synonymous


In [159]:
df_allele_name = ds_linreg.variant_allele.to_pandas().drop([2,3], axis=1).rename(columns={0: 'ref', 1: 'alt'})
df_allele_count = ds_linreg.pipe(sg.variant_stats).variant_allele_count.to_pandas().drop([2,3], axis=1).rename(columns={0: 'ref_count', 1: 'alt_count'})
df_allele = pd.merge(df_allele_name,df_allele_count, on='variants')
bigtable = pd.merge(df_other, df_allele, on='variants')
# bigtable[[]]
bigtable = pd.merge(df_other, df_geno, on='variants')
bigtable['mutation'].str.split().str[0].str[0] == bigtable['mutation'].str.split().str[0].str[-1]

variants
ID_5_C          False
ID_34_C          True
ID_60_G          True
ID_63_T          True
ID_84_A          True
                ...  
ID_1830660_G    False
ID_1830678_A    False
ID_1830680_T    False
ID_1830686_T    False
ID_1830689_C    False
Name: mutation, Length: 147369, dtype: bool

In [117]:
ds_linreg.pipe(sg.variant_stats)

<xarray.Dataset>
Dimensions:                   (variants: 147369, alleles: 4, samples: 262,
                               ploidy: 1, contigs: 1, filters: 2)
Coordinates:
  * variants                  (variants) object 'ID_5_C' ... 'ID_1830689_C'
Dimensions without coordinates: alleles, samples, ploidy, contigs, filters
Data variables: (12/38)
    variant_n_called          (variants) int64 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_call_rate         (variants) float64 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_n_het             (variants) int64 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_n_hom_ref         (variants) int64 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_n_hom_alt         (variants) int64 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_n_non_ref         (variants) int64 dask.array<chunksize=(5628,), meta=np.ndarray>
    ...                        ...
    serotype                  (samples) object dask.array<chunksize=(262,), meta=np.ndarray>
    variant_allele            (variants, alleles) object dask.array<chunksize=(5628, 4), meta=np.ndarray>
    variant_contig            (variants) int8 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_filter            (variants, filters) bool dask.array<chunksize=(5628, 2), meta=np.ndarray>
    variant_position          (variants) int32 dask.array<chunksize=(5628,), meta=np.ndarray>
    variant_quality           (variants) float32 dask.array<chunksize=(5628,), meta=np.ndarray>

In [ ]:
ds_mapping = (pd.read_csv('results/mapping.csv')
              .to_xarray()
              .reset_index('index').reset_coords('index').rename_dims({'index' : 'variants'})
              .rename_vars({'id': 'variant_id'}).set_coords('variant_id').set_index({'variants' : 'variant_id'})
              .drop_vars(['index', 'Unnamed: 0'])
)
ds_mic = (sg.load_dataset('results/zarrs/Hinf_norm_mic.zarr/')
          .assign(variant_id=lambda ds: ( ds.variant_position.to_series().astype(str) + '_' + ds.variant_allele[:,1].to_series()))
          .set_coords('variant_id').set_index({'variants' : 'variant_id'})
)
ds_final = ds_mic.merge(ds_mapping, join='left')
ds_final.sel(variants=((1688288 < ds_final.variant_position) & (ds_final.variant_position < 1690120)).compute())
ds_final

In [ ]:
baba = ds_final[['variant_position', 'mutation']].to_pandas()
baba.query('(1688288 < variant_position < 1690120) & (mutation.str.contains("N526K"))')

In [ ]:
ds_final.sel(variants=(ds_final.variant_position == 1688543)).variant_allele.values

In [ ]:
mapping.query('1688288 < position < 1690120')

In [ ]:
ds_linreg.sel(variants=(ds_linreg.variants == 'ID_1688543_T')).call_genotype

In [ ]:
csv = pd.read_feather('results/Hflu_association_data_cf4_cr4_fr75_ph8_l0_x0_271_combined_amended_u95_phylo.feather')

csv.query(' position == 903617')

In [ ]:
ds_bla = sg.load_dataset('results/zarrs/Hinf_norm_mic_temp.zarr/')
# ds_bla.sel(samples=(ds_bla.serotype == 'Serotype e').compute()).sample_id.values
ds_bla.serotype.values

In [ ]:
dddd = pd.read_feather('results/feather/Hinf_norm_mic.feather')

In [ ]:
dddd[dddd.serotype == 'Serotype e']

In [ ]:
import pandas as pd

mash = pd.read_csv('results/mash.tsv', sep='\t', index_col=0)
mash.index = mash.index.str.split('_').str[0]
mash.columns = mash.columns.str.split('_').str[0]
mash